# Transformers 模型量化技术：GPTQ

![image](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/159_autogptq_transformers/thumbnail.jpg)

2022年，Frantar等人发表了论文 [GPTQ：Accurate Post-Training Quantization for Generative Pre-trained Transformers](https://arxiv.org/abs/2210.17323)。

这篇论文详细介绍了一种训练后量化算法，适用于所有通用的预训练 Transformer模型，同时只有微小的性能下降。

GPTQ算法需要通过对量化模型进行推理来校准模型的量化权重。详细的量化算法在[原始论文](https://arxiv.org/pdf/2210.17323.pdf)中有描述。

基于[`auto-gptq`](https://github.com/PanQiWei/AutoGPTQ.git) 开源实现库，transformers 支持使用GPTQ算法量化的模型。

## 使用 GPTQ 量化模型

为了使用 `auto-gptq` 库量化一个模型，我们需要向量化器传递一个数据集。

通常有两种方式构造数据集：
- 量化器支持的默认数据集（包括`['wikitext2','c4','c4-new','ptb','ptb-new']`）
- 一个字符串列表（这些字符串将被用作数据集）

### 使用 GPTQ 算法支持的默认数据集来量化

在下面的示例中，让我们尝试使用`"wikitext2"`数据集将模型量化为4位精度。支持的精度有`[2, 4, 6, 8]`。

In [1]:
from transformers import GPTQConfig,AutoModelForCausalLM

model_name_or_path = "facebook/opt-6.7b"

#定义GPTQ 量化参数
quantization_config = GPTQConfig(
    bits=4,
    dataset="wikitext2",
    group_size=64,
    desc_act=False
)

/root/autodl-tmp/Anaconda3/envs/peft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [3]:
import os
%env HF_HOME=/root/autodl-tmp/huggingface/cache
%env HF_ENDPOINT = https://hf-mirror.com
%env HF_HUB_CACHE = /root/autodl-tmp/huggingface/cache
os.environ['HF_HOME'] = '/root/autodl-tmp/huggingface/cache'
os.environ['HF_HUB_CACHE'] = '/root/autodl-tmp/huggingface/cache'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'


env: HF_HOME=/root/autodl-tmp/huggingface/cache
env: HF_ENDPOINT=https://hf-mirror.com
env: HF_HUB_CACHE=/root/autodl-tmp/huggingface/cache


In [4]:
#加载量化模型
quant_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=quantization_config,
    device_map = 'auto'
)

/root/autodl-tmp/Anaconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
CUDA extension not installed.
CUDA extension not installed.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.68s/it]
/root/autodl-tmp/Anaconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/autodl-tmp/Anaconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to for

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 